<a href="https://colab.research.google.com/github/farbodyadollahi8081/Deep-Learning/blob/main/HW3/Q2/Knowladge_Distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning HW3 question 2 Knowladge Distillation

Student Name: Farbod Yadollahi

Student Number: 98102595

In [5]:
import time
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import datasets, transforms
from torchsummary import summary
from torch.optim import lr_scheduler
import torch.nn.functional as F
import torch.nn as nn
import torchvision.models as models
from torch import nn, optim

loading CIFAR10 dataset

In [6]:
transform = transforms.Compose([transforms.Resize((224,224)),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485,0.456,  
                                0.406], [0.229, 0.224, 0.225])])
trainset = datasets.CIFAR10('/content/train/', download=True, train=True, transform=transform)
valset = datasets.CIFAR10('/content/val/', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True)
len_trainset = len(trainset)
len_valset = len(valset)
classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

Files already downloaded and verified
Files already downloaded and verified
cpu


### Part A: Teacher model Preparation

In [7]:
resnet = models.resnet50(pretrained=True)
for param in resnet.parameters():
   param.requires_grad = False
numClasses = resnet.fc.in_features
resnet.fc = nn.Linear(numClasses, 10)
resnet = resnet.to(device)
lossFunction = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.fc.parameters())

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [8]:
def trainTest(model, trainloader, valloader, lossFunction, optimizer, len_trainset, len_valset, num_epochs=25):
   model.train()
   best_model_wts = copy.deepcopy(model.state_dict())
   best_acc = 0.0
   for epoch in range(num_epochs):
      model.train()
      print('Epoch {}/{}'.format(epoch, num_epochs-1))
      print('-' * 10)
      running_loss = 0.0
      running_corrects = 0
      for inputs, labels in trainloader:
         inputs = inputs.to(device)
         labels = labels.to(device)
         optimizer.zero_grad()
         outputs = model(inputs)
         loss = lossFunction(outputs, labels)
         _, preds = torch.max(outputs, 1)
         loss.backward() 
         optimizer.step()  
         running_loss += loss.item() * inputs.size(0)
         running_corrects += torch.sum(preds == labels.data)
      epoch_loss = running_loss / len_trainset
      epoch_acc = running_corrects.double() / len_trainset
      print(' Train Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss,
             epoch_acc)) 
         
      model.eval()
      running_loss_val = 0.0 
      running_corrects_val = 0
      for inputs, labels in valloader:
         inputs = inputs.to(device)
         labels = labels.to(device)
         outputs = model(inputs) 
         loss = lossFunction(outputs,labels)
         _, preds = torch.max(outputs, 1)
         running_loss_val += loss.item() * inputs.size(0)
         running_corrects_val += torch.sum(preds == labels.data)
      
      epoch_loss_val = running_loss_val / len_valset
      epoch_acc_val = running_corrects_val.double() / len_valset
      
      if epoch_acc_val > best_acc:
         best_acc = epoch_acc_val
         best_model_wts = copy.deepcopy(model.state_dict())
      
      print(' Val Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss_val,
             epoch_acc_val))
      
      print()
      print('Best val Acc: {:4f}'.format(best_acc))
      model.load_state_dict(best_model_wts)
   return model

In [ ]:
resnet_teacher = trainTest(resnet,trainloader,
                                   valloader,lossFunction,optimizer,
                                   len_trainset,len_valset,10)

Epoch 0/9
----------
 Train Loss: 0.7595 Acc: 0.7469
 Val Loss: 0.5872 Acc: 0.7991

Best val Acc: 0.799100
Epoch 1/9
----------
 Train Loss: 0.5928 Acc: 0.7950
 Val Loss: 0.5550 Acc: 0.8073

Best val Acc: 0.807300
Epoch 2/9
----------
 Train Loss: 0.5595 Acc: 0.8061
 Val Loss: 0.5583 Acc: 0.8065

Best val Acc: 0.807300
Epoch 3/9
----------
 Train Loss: 0.5688 Acc: 0.8041
 Val Loss: 0.5427 Acc: 0.8148

Best val Acc: 0.814800
Epoch 4/9
----------
 Train Loss: 0.5465 Acc: 0.8097
 Val Loss: 0.5344 Acc: 0.8168

Best val Acc: 0.816800
Epoch 5/9
----------
 Train Loss: 0.5282 Acc: 0.8168
 Val Loss: 0.5355 Acc: 0.8161

Best val Acc: 0.816800
Epoch 6/9
----------
 Train Loss: 0.5350 Acc: 0.8149
 Val Loss: 0.5428 Acc: 0.8147

Best val Acc: 0.816800
Epoch 7/9
----------
 Train Loss: 0.5309 Acc: 0.8157
 Val Loss: 0.5250 Acc: 0.8235

Best val Acc: 0.823500
Epoch 8/9
----------
 Train Loss: 0.5225 Acc: 0.8187
 Val Loss: 0.5081 Acc: 0.8241

Best val Acc: 0.824100
Epoch 9/9
----------
 Train Loss: 0.5

best acc =0.8241

### Part B: Training Student with the Teacher

In [9]:
student_model=models.resnet18()
NCS = student_model.fc.in_features  #number of classes in student model
student_model.fc = nn.Linear(NCS, 10)
student_model = student_model.to(device)


In [10]:
def loss_kd(outputs, labels, teacher_outputs, temparature, alpha):
   KD_loss = nn.KLDivLoss()(F.log_softmax(outputs/temparature, 
             dim=1),F.softmax(teacher_outputs/temparature,dim=1)) * (alpha * temparature * temparature) + F.cross_entropy(outputs, labels) * (1. - alpha)
   return KD_loss
def get_outputs(model, dataloader):
   outputs = []
   for inputs, labels in dataloader:
      inputs_batch, labels_batch = inputs.cuda(), labels.cuda()
      output_batch = model(inputs_batch).data.cpu().numpy()
      outputs.append(output_batch)
   return outputs


In [11]:
def train_kd(model,teacher_out, optimizer, loss_kd, dataloader, temparature, alpha):
   model.train()
   running_loss = 0.0
   running_corrects = 0
   for i,(images, labels) in enumerate(dataloader):
      inputs = images.to(device)
      labels = labels.to(device)
      optimizer.zero_grad()
      outputs = model(inputs)
      outputs_teacher = torch.from_numpy(teacher_out[i]).to(device)
      loss = loss_kd(outputs,labels,outputs_teacher,temparature, 
                     alpha)
      _, preds = torch.max(outputs, 1)
      loss.backward()
      optimizer.step()
      running_loss += loss.item() * inputs.size(0)
      running_corrects += torch.sum(preds == labels.data)
   
   epoch_loss = running_loss / len(trainset)
   epoch_acc = running_corrects.double() / len(trainset)
   print('Train Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, 
          epoch_acc))
def eval_kd(model,teacher_out, optimizer, loss_kd, dataloader, temparature, alpha):
   model.eval()
   running_loss = 0.0
   running_corrects = 0
   for i,(images, labels) in enumerate(dataloader):
      inputs = images.to(device)
      labels = labels.to(device)
      outputs = model(inputs)
      outputs_teacher = torch.from_numpy(teacher_out[i]).cuda()
      loss = loss_kd(outputs,labels,outputs_teacher,temparature, 
                     alpha)
      _, preds = torch.max(outputs, 1)
      running_loss += loss.item() * inputs.size(0)
      running_corrects += torch.sum(preds == labels.data)
   epoch_loss = running_loss / len(valset)
   epoch_acc = running_corrects.double() / len(valset)
   print(' Val Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss,
          epoch_acc))
   return epoch_acc
def train_and_evaluate_kd(model, teacher_model, optimizer, loss_kd, trainloader, valloader, temparature, alpha, num_epochs=25):
   teacher_model.eval()
   best_model_wts = copy.deepcopy(model.state_dict())
   outputs_teacher_train = get_outputs(teacher_model, trainloader)
   outputs_teacher_val = get_outputs(teacher_model, valloader)
   print("Teacher's outputs are computed now starting the training process-")
   best_acc = 0.0
   for epoch in range(num_epochs):
      print('Epoch {}/{}'.format(epoch, num_epochs -1))
      print('-' * 10)
      
      # Training the student with the soft labes as the outputs from the teacher and using the loss_kd function
      
      train_kd(model, outputs_teacher_train, 
               optim.Adam(student_model.parameters()),loss_kd,trainloader, 
               temparature, alpha)
     
      # Evaluating the student network
      epoch_acc_val = eval_kd(model, outputs_teacher_val, optim.Adam(student_model.parameters()), loss_kd, valloader, temparature, alpha)
      if epoch_acc_val > best_acc:
         best_acc = epoch_acc_val
         best_model_wts = copy.deepcopy(model.state_dict())
         print('Best val Acc: {:4f}'.format(best_acc))
         model.load_state_dict(best_model_wts)
   return model

In [ ]:
stud=train_and_evaluate_kd(student_model,resnet_teacher, optim.Adam(student_model.parameters()),loss_kd,trainloader,valloader,1,0.5,20)

Teacher's outputs are computed now starting the training process-
Epoch 0/19
----------
Train Loss: 0.4215 Acc: 0.8651
 Val Loss: 0.4657 Acc: 0.8236
Best val Acc: 0.823600
Epoch 1/19
----------
Train Loss: 0.3796 Acc: 0.9002
 Val Loss: 0.4612 Acc: 0.8323
Best val Acc: 0.832300
Epoch 2/19
----------
Train Loss: 0.3430 Acc: 0.9316
 Val Loss: 0.4417 Acc: 0.8444
Best val Acc: 0.844400
Epoch 3/19
----------
Train Loss: 0.3111 Acc: 0.9549
 Val Loss: 0.4660 Acc: 0.8348
Epoch 4/19
----------
Train Loss: 0.2877 Acc: 0.9748
 Val Loss: 0.4349 Acc: 0.8548
Best val Acc: 0.854800
Epoch 5/19
----------
Train Loss: 0.2743 Acc: 0.9828
 Val Loss: 0.4592 Acc: 0.8314
Epoch 6/19
----------
Train Loss: 0.2651 Acc: 0.9886
 Val Loss: 0.4315 Acc: 0.8562
Best val Acc: 0.856200
Epoch 7/19
----------
Train Loss: 0.2569 Acc: 0.9921
 Val Loss: 0.4286 Acc: 0.8574
Best val Acc: 0.857400
Epoch 8/19
----------
Train Loss: 0.2533 Acc: 0.9938
 Val Loss: 0.4324 Acc: 0.8550
Epoch 9/19
----------
Train Loss: 0.2507 Acc: 0.9

best acc = 0.8734

### Part C: Training RESNET18 without the Teacher

In [12]:
NNwithoutTeacher = models.resnet18()
noFTRS = NNwithoutTeacher.fc.in_features
NNwithoutTeacher.fc = nn.Linear(noFTRS, 10)
NNwithoutTeacher =  NNwithoutTeacher.to(device)
lossFunction = nn.CrossEntropyLoss()
optimizer2 = optim.Adam(NNwithoutTeacher.parameters())

In [ ]:
New_resnet_18 = trainTest(NNwithoutTeacher,trainloader,
                                   valloader,lossFunction,optimizer2,
                                   len_trainset,len_valset,10)

Epoch 0/9
----------
 Train Loss: 1.3268 Acc: 0.5166
 Val Loss: 1.1159 Acc: 0.6142

Best val Acc: 0.614200
Epoch 1/9
----------
 Train Loss: 0.7994 Acc: 0.7186
 Val Loss: 0.7644 Acc: 0.7311

Best val Acc: 0.731100
Epoch 2/9
----------
 Train Loss: 0.5987 Acc: 0.7910
 Val Loss: 0.8245 Acc: 0.7155

Best val Acc: 0.731100
Epoch 3/9
----------
 Train Loss: 0.6109 Acc: 0.7848
 Val Loss: 0.6539 Acc: 0.7744

Best val Acc: 0.774400
Epoch 4/9
----------
 Train Loss: 0.4760 Acc: 0.8340
 Val Loss: 0.6287 Acc: 0.7840

Best val Acc: 0.784000
Epoch 5/9
----------
 Train Loss: 0.3722 Acc: 0.8689
 Val Loss: 0.5270 Acc: 0.8267

Best val Acc: 0.826700
Epoch 6/9
----------
 Train Loss: 0.2886 Acc: 0.8992
 Val Loss: 0.5351 Acc: 0.8220

Best val Acc: 0.826700
Epoch 7/9
----------
 Train Loss: 0.2950 Acc: 0.8966
 Val Loss: 0.5912 Acc: 0.8179

Best val Acc: 0.826700
Epoch 8/9
----------
 Train Loss: 0.2968 Acc: 0.8964
 Val Loss: 0.5190 Acc: 0.8288

Best val Acc: 0.828800
Epoch 9/9
----------
 Train Loss: 0.2

best acc =0.8375

as it is obvious, the accuracy in the case without teacher is a bit less than the case when we use the teacher. because by the use of pretrained bigger teacher model, we train the student model to be accustomed to the complexity of data. while raw training frustrates the student model to be well trained on the complexity of data.

### Part D: Full Fine Tuning

In [13]:
New_Teacher = models.resnet50()
noFTRS1 = New_Teacher.fc.in_features
New_Teacher.fc = nn.Linear(noFTRS1, 10)
New_Teacher = New_Teacher.to(device)
lossFunction = nn.CrossEntropyLoss()
params_1x = [param for name, param in New_Teacher.named_parameters() if 'fc' not in str(name)]
optimizer2 = optim.Adam([{'params':params_1x}, {'params': New_Teacher.fc.parameters()}])

In [ ]:
resnet_new_teacher = trainTest(New_Teacher,trainloader,
                                   valloader,lossFunction,optimizer2,
                                   len_trainset,len_valset,1)

Epoch 0/0
----------


after 6 hours and  half being run, unfortunately the resnet 50 couldn't be trained. but it is obvious that with the full fine tuning student model will perform better. since the teacher model hasn't trained successfully, the student model can't be trained either. the training code for the student model ,though has been provided below. 

best acc =0.8319

In [ ]:
student_model2=models.resnet18()
NCS2 = student_model2.fc.in_features  #number of classes in student model
student_model2.fc = nn.Linear(NCS2, 10)
student_model2 = student_model2.to(device)

In [ ]:
stud2=train_and_evaluate_kd(student_model2,resnet_new_teacher, optim.Adam(student_model2.parameters()),loss_kd,trainloader,valloader,1,0.5,20)

it is obvious that having resnet 50 fully fine tuned on cifar 10, the weights are tuned well, so the dark knowladge can be better transfered to the student model while we use knowladge distillation. so we expect to have better perfomance of the student model afterwards. 